In [1]:
from IPython.display import clear_output

Clone repository

In [2]:
!git clone https://github.com/7embl4/asr.git asr
%cd asr

Cloning into 'asr'...
remote: Enumerating objects: 366, done.
remote: Counting objects: 100% (366/366), done.
remote: Compressing objects: 100% (222/222), done.
remote: Total 366 (delta 185), reused 297 (delta 116), pack-reused 0 (from 0)
Receiving objects: 100% (366/366), 383.82 KiB | 31.98 MiB/s, done.
Resolving deltas: 100% (185/185), done.
/content/asr


Install all required libs

In [3]:
!pip install -r requirements.txt
clear_output()

Load model

In [4]:
!huggingface-cli download artem1085715/conformer-small --local-dir 'model/'
clear_output()

Load dataset

In [5]:
!gdown 1kyESoLRy5oCGE7y_vFUbNZxQenxfHigd
!unzip test_data.zip -d .
!rm test_data.zip

Downloading...
From: https://drive.google.com/uc?id=1kyESoLRy5oCGE7y_vFUbNZxQenxfHigd
To: /content/asr/test_data.zip
100% 746k/746k [00:00<00:00, 87.3MB/s]
Archive:  test_data.zip
   creating: ./test_data/
   creating: ./test_data/audio/
  inflating: ./test_data/audio/84-121550-0000.flac  
  inflating: ./test_data/audio/84-121550-0001.flac  
  inflating: ./test_data/audio/84-121550-0002.flac  
  inflating: ./test_data/audio/84-121550-0003.flac  
  inflating: ./test_data/audio/84-121550-0004.flac  
   creating: ./test_data/transcriptions/
  inflating: ./test_data/transcriptions/84-121550-0000.txt  
  inflating: ./test_data/transcriptions/84-121550-0001.txt  
  inflating: ./test_data/transcriptions/84-121550-0002.txt  
  inflating: ./test_data/transcriptions/84-121550-0003.txt  
  inflating: ./test_data/transcriptions/84-121550-0004.txt  


Run inference. You have to provide:
- path to model you use
- config for a custom datasets
- paths to audio and it's transcriptions
- *optional:* if your dataset has less elements than default batch size of dataloader (8 by default), you should specify it in hydra parameters too

Note: Your custom dataset should follow this structure as shown below
```bash
dataset_name
|-- audio
|   |-- audio1.wav  # might be .flac
|   |-- audio2.wav
|   |-- ...
|-- transcriptions
|   |-- transcription1.txt
|   |-- transcription2.txt
|   |-- ...
```

In [6]:
!python inference.py \
        inferencer.from_pretrained="model/model_best.pth" \
        dataloader.batch_size=5 \
        datasets="custom" \
        datasets.test.audio_dir="test_data/audio" \
        datasets.test.transcription_dir="test_data/transcriptions"

CTCModel(
  (subsampling): Subsampling(
    (conv): Conv1d(80, 80, kernel_size=(2,), stride=(2,))
    (linear): Linear(in_features=80, out_features=144, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (conf_blocks): Sequential(
    (0): ConformerBlock(
      (layers): Sequential(
        (0): FeedForwardModule(
          (layers): Sequential(
            (0): LayerNorm((144,), eps=1e-05, elementwise_affine=True)
            (1): Linear(in_features=144, out_features=576, bias=True)
            (2): SiLU()
            (3): Dropout(p=0.1, inplace=False)
            (4): Linear(in_features=576, out_features=144, bias=True)
            (5): Dropout(p=0.1, inplace=False)
          )
        )
        (1): MultiHeadAttentionModule(
          (layers): Sequential(
            (0): LayerNorm((144,), eps=1e-05, elementwise_affine=True)
            (1): TransformerEncoder(
              (layers): ModuleList(
                (0-15): 16 x TransformerEncoderLayer(
                  (se